# Task II: Classical GNNs #

The model chosen for this task is a Point Net graph neural network (GNN). This was done with the consideration that the raw data is a point cloud of quark and gluon jets. The Point Net architecture is specifically designed to implement a GNN on such data.

This is one of my first data extraction methods when I was trying to understand the data format and representation. I didn't end up using it directly, but it helped me visualize the data storage structure.

In [12]:
import numpy as np

data = np.load('qg_dataset/QG_jets.npz', allow_pickle=True)
DATA_SIZE = 0
for key, value in data.items():
    print(key, value)
    DATA_SIZE = len(value)

X [[[ 2.68769142e-01  3.56903171e-01  4.74138734e+00  2.20000000e+01]
  [ 1.60076377e-01 -2.55609533e-01  4.55022910e+00  2.20000000e+01]
  [ 1.14868731e+00 -6.24380156e-02  4.50385377e+00 -2.11000000e+02]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 1.21266321e+00 -1.12853089e-01  3.04708757e+00 -2.11000000e+02]
  [ 2.40893976e-01 -1.67174886e-02  2.82705667e+00  2.20000000e+01]
  [ 1.02778452e-01 -8.58720522e-02  3.04180579e+00  2.20000000e+01]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 2.16829416e-01 -9.97057017e-01  5.32568913e-01  2.20000000e+01]
  [ 2.31359397e-01 -1.59192211e+00  2.02906587e-01  2.20000000e+01]
  [ 3.41572501e-01 -1.34588077

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import numpy as np
import matplotlib as plt

Here is the core of the Point Net architecture. It uses 


In [16]:
class CustomNet(nn.Module):
    def __init__(self, input_channels=4):
        super(CustomNet, self).__init__()
        
        self.conv1 = nn.Conv1d(input_channels, 32, 1)
        self.conv2 = nn.Conv1d(32, 64, 1)
        self.conv3 = nn.Conv1d(64, 64, 1)
        self.conv4 = nn.Conv1d(64, 128, 1)
        self.conv5 = nn.Conv1d(128, 256, 1)
        
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.relu(self.conv5(x))
        
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 256)
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.softmax(x)
        
        return x

Originally, I had higher dimension arguments with two QG datasets (two .npz files), but the training was taking 10 minutes/epoch. I limited the dataset to one file, downscaled the network, increased the batch size, and decreased the number of epochs from the original parameters.


In [18]:
class JetDataset(torch.utils.data.Dataset):
    def __init__(self, data_path):
        self.data = np.load(data_path, allow_pickle=True)
        self.samples = []
        
        for key, value in self.data.items():
            if key == 'X':
                self.samples.extend([(sample, label) for sample, label in zip(value, self.data['y'])])
                
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample, label = self.samples[idx]
        sample = torch.tensor(sample.transpose(), dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long) 
        return sample, label

data_path = 'qg_dataset/QG_jets.npz'
jet_dataset = JetDataset(data_path)
batch_size = 128
data_loader = DataLoader(jet_dataset, batch_size=batch_size, shuffle=True)

This is the model initialization and training section.

In [19]:
model = CustomNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

num_epochs = 3

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


: 